In [1]:
import time
import multiprocessing
import asyncio




# define my async worker function
async def worker1():
    
    num=0
    while True:
        num+=1
        print(num)
        time.sleep(1)
        await asyncio.sleep(1)

        
# define the poison-pill function
async def kill_loop(loop,event):
    while True:
        print(event.is_set())
        if event.is_set():
            loop.stop()
        await asyncio.sleep(1)
        
async def print_something():
    while True:
        print('test')
        await asyncio.sleep(1)


# so .. this is the loop...
def starting_up_the_loop(event):
    loop = asyncio.get_event_loop()
    loop.create_task(worker1())
    loop.create_task(kill_loop(loop,event))
    loop.create_task(print_something())
    loop.run_forever()
    
    

# so -- make a separate process, then.
event=multiprocessing.Event()

P=multiprocessing.Process(target=starting_up_the_loop,args=(event,))
        
    

## Let's Start

In [2]:
P

<Process(Process-1, initial)>

In [3]:
event.is_set()

False

In [4]:
P.start()

1
False
test
2
False
test
3
True


## The event Loop within the Process started - let's shut it down

In [5]:
event.set()

## It stopped! --> Clean exit!